In [1]:
"""Getting Started Example for Python 2.7+/3.3+"""
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir
import json
import wave
import pyaudio

In [2]:
config_json = open('../config.json')
config_dict = json.load(config_json)

# Create a client using the credentials and region defined in the [adminuser]
# section of the AWS credentials file (~/.aws/credentials).
session = Session(aws_access_key_id=config_dict["AWS_ACCESS_KEY_ID"],
                  aws_secret_access_key=config_dict["AWS_SECRET_ACCESS_KEY"])
polly = session.client("polly", region_name='eu-west-1')

In [17]:
my_text = '''
<speak>
Hola otra vez <break time="1s"/> la voz de antes era un poco mala pero yo soy mejor.
</speak>
'''

In [18]:
response = polly.synthesize_speech(
    Text=my_text, 
    OutputFormat="mp3", 
    VoiceId="Lucia",
    SampleRate="16000",
    Engine="neural",
    TextType="ssml"
)

In [19]:
with closing(response["AudioStream"]) as stream:
    # output = os.path.join(gettempdir(), "speech.mp3")
    with open("speech.mp3", "wb") as file:
           file.write(stream.read())
            

In [20]:
# Initializing variables
CHANNELS = 1  # Polly's output is a mono audio stream
RATE = 44100
WAV_SAMPLE_WIDTH_BYTES = 2  # Polly's output is a stream of 16-bits (2 bytes) samples
FRAMES = []

# Processing the response to audio stream
STREAM = response.get("AudioStream")
FRAMES.append(STREAM.read())

WAVE_FORMAT = wave.open("speech.wav", 'wb')
# WAVE_FORMAT = wave.open(OUTPUT_FILE_IN_WAVE, 'wb')
WAVE_FORMAT.setnchannels(CHANNELS)
WAVE_FORMAT.setsampwidth(WAV_SAMPLE_WIDTH_BYTES)
WAVE_FORMAT.setframerate(RATE)
WAVE_FORMAT.writeframes(b''.join(FRAMES))
WAVE_FORMAT.close()

In [21]:
CHUNK = 1024

# open a wav format music
f = wave.open("speech.wav", "rb")
# instantiate PyAudio
p = pyaudio.PyAudio()
# open stream
stream = p.open(format=p.get_format_from_width(f.getsampwidth()),
                channels=f.getnchannels(),
                rate=f.getframerate(),
                output=True)
# read data
data = f.readframes(CHUNK)

# play stream
while data:
    stream.write(data)
    data = f.readframes(CHUNK)

    # stop stream
stream.stop_stream()
stream.close()

# close PyAudio
p.terminate()